In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from funs import *

In [2]:
from sklearn.metrics import accuracy_score, classification_report

In [3]:
df = pd.read_csv('../data/df_train.csv')
df_test = pd.read_csv('../data/df_test.csv')

### Najpierw wybieramy tylko niektóre kolumny

In [4]:
columns_to_drop = ["Area", "MinorAxisLength", "MajorAxisLength", "ConvexArea", "EquivDiameter"]
df = df.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)


### Zamieniamy outliery

In [5]:
X = df.copy()

mean = X.mean()
std = X.std()
cut_off = std * 3
lower, upper = mean - cut_off, mean + cut_off
outliers = (X < lower) | (X > upper)
X = X.where(~outliers, mean, axis=1)

X_train = X.drop(columns=["Class"])
y_train = X["Class"]
X_test = df_test.drop(columns=["Class"])
y_test = df_test["Class"]

C:\Users\Tomek\AppData\Local\Temp\ipykernel_6464\4181064245.py:3: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  median = X.median()
C:\Users\Tomek\AppData\Local\Temp\ipykernel_6464\4181064245.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean = X.mean()
C:\Users\Tomek\AppData\Local\Temp\ipykernel_6464\4181064245.py:5: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid col

In [6]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=80, criterion='entropy', max_depth=20, random_state=311, max_features=None,
                             warm_start=True)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.925148653375306
              precision    recall  f1-score   support

    BARBUNYA       0.93      0.92      0.92       278
      BOMBAY       0.99      1.00      1.00       109
        CALI       0.94      0.94      0.94       342
    DERMASON       0.91      0.93      0.92       745
       HOROZ       0.96      0.93      0.95       405
       SEKER       0.95      0.94      0.95       426
        SIRA       0.88      0.88      0.88       554

    accuracy                           0.93      2859
   macro avg       0.94      0.93      0.94      2859
weighted avg       0.93      0.93      0.93      2859



In [13]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(estimator=rfc, random_state=311, learning_rate=0.2,
                           n_estimators=21, algorithm='SAMME')
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9265477439664218
              precision    recall  f1-score   support

    BARBUNYA       0.93      0.92      0.93       278
      BOMBAY       0.99      1.00      1.00       109
        CALI       0.94      0.95      0.95       342
    DERMASON       0.91      0.93      0.92       745
       HOROZ       0.97      0.93      0.95       405
       SEKER       0.95      0.94      0.95       426
        SIRA       0.88      0.87      0.87       554

    accuracy                           0.93      2859
   macro avg       0.94      0.94      0.94      2859
weighted avg       0.93      0.93      0.93      2859

